# Gemma-3-1B-it RAG System — Google Colab

**Vor dem Start:** Oben rechts auf `Laufzeit` → `Laufzeittyp ändern` → **T4 GPU** auswählen!

Dann alle Zellen von oben nach unten ausführen.

In [ ]:
# Zelle 1: GPU prüfen
import torch
print('CUDA verfügbar:', torch.cuda.is_available())
print('GPU:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'KEINE GPU — Laufzeittyp auf T4 setzen!')

In [ ]:
# Zelle 2: Abhängigkeiten installieren
!pip install transformers accelerate bitsandbytes sentence-transformers chromadb fastapi
!pip install --upgrade wandb -q
uvicorn gradio pydantic-settings python-dotenv requests wandb --upgrade -q

In [ ]:
# Zelle 3: GitHub Repository klonen
!git clone https://github.com/SebastianKuehnrich/gemma3-rag-api.git
%cd gemma3-rag-api/Deployment2
!ls

In [ ]:
# Zelle 4: HF Token aus Colab Secrets laden (Gemma ist ein gated model)
from google.colab import userdata
import os

hf_token = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = hf_token

# .env Datei schreiben damit pydantic-settings sie findet
with open('.env', 'w') as f:
    f.write(f"HF_TOKEN={hf_token}\n")
    f.write("USE_4BIT_QUANTIZATION=True\n")
    f.write("MAX_NEW_TOKENS=200\n")
print('.env geschrieben')

In [ ]:
# Zelle 5: Datenbank initialisieren
!python -m scripts.init_database

In [ ]:
# Zelle 6: FastAPI im Hintergrund starten
import threading, subprocess, time, requests

def run_fastapi():
    subprocess.run(['uvicorn', 'app.main:app', '--host', '0.0.0.0', '--port', '8001'])

thread = threading.Thread(target=run_fastapi, daemon=True)
thread.start()

# Warten bis FastAPI bereit ist (Gemma laden dauert ~30-60s auf T4)
print('Warte auf FastAPI + Gemma...')
for i in range(120):
    try:
        r = requests.get('http://127.0.0.1:8001/health', timeout=2)
        if r.status_code == 200:
            data = r.json()
            print(f'FastAPI bereit! Gemma geladen: {data["gemma_loaded"]}')
            print(f'Dokumente in DB: {data["num_documents"]}')
            break
    except:
        pass
    time.sleep(2)
    if i % 10 == 0:
        print(f'  ... {i*2}s vergangen')
else:
    print('TIMEOUT — FastAPI hat nicht gestartet')

In [ ]:
# Zelle 7: Schneller API-Test direkt in Python
import requests, json

response = requests.post(
    'http://127.0.0.1:8001/query',
    json={'query': 'Was ist Machine Learning?', 'top_k': 3, 'use_rag': True},
    timeout=60
)
data = response.json()
print('Antwort:', data['answer'])
print('Generierungszeit:', data['generation_time_seconds'], 's')
print('Kontext-Dokumente:', data['num_context_docs'])

In [ ]:
# Zelle 8: Gradio UI mit öffentlicher URL starten
import ui.gradio_app as gradio_module
from ui.gradio_app import demo

# API URL auf lokalen FastAPI Server setzen
gradio_module.API_URL = 'http://127.0.0.1:8001'

# Gradio starten — share=True erzeugt automatisch eine öffentliche URL
demo.launch(
    server_name='0.0.0.0',
    server_port=7860,
    share=True,
)

---
## BONUS: Fine-Tuning mit wandb (optional)

Diese Zelle startet das Fine-Tuning auf der T4 GPU. Dauert ~15-30 Minuten.

In [ ]:
# Zelle 9 (OPTIONAL): wandb Login + Fine-Tuning starten
from google.colab import userdata
import os

wandb_key = userdata.get('WANDB_API_KEY')
os.environ['WANDB_API_KEY'] = wandb_key

# .env updaten
with open('.env', 'a') as f:
    f.write(f"WANDB_API_KEY={wandb_key}\n")

!python fine_tune.py